<a href="https://colab.research.google.com/github/Shamiul-693/-Exploratory-Data-Analysis-and-Classification-Algorithms-on-Mushroom-Dataset/blob/main/Sherlock_Holmes_dataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

In [2]:
import os

def load_dataset(folder_path):
    stories = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                stories.append(file.read())
    return stories

# Your extraction path
folder_path = "/content/drive/MyDrive/ Sherlock /sherlock"
stories = load_dataset(folder_path)

# Check the number of stories loaded
print(f"Loaded {len(stories)} stories.")


Loaded 67 stories.


In [6]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Ensure required NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
# Download the 'punkt_tab' resource
nltk.download('punkt_tab') # This line downloads the missing resource.

# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize whitespace
    tokens = word_tokenize(text)  # Tokenize
    stop_words = set(stopwords.words('english'))  # Load stopwords
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return " ".join(tokens)

# Apply preprocessing to all stories
cleaned_stories = [preprocess_text(story) for story in stories]

# Example: Print the first 500 characters of the first cleaned story
print(cleaned_stories[0][:500])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


adventure beryl coronet arthur conan doyle holmes said stood one morning bowwindow looking street madman coming along seems rather sad relatives allow come alone friend rose lazily armchair stood hands pockets dressinggown looking shoulder bright crisp february morning snow day still lay deep upon ground shimmering brightly wintry sun centre baker street ploughed brown crumbly band traffic either side heapedup edges footpaths still lay white fell grey pavement cleaned scraped still dangerously s


In [7]:
summaries = [
    "Summary of story 1",
    "Summary of story 2",
    # Add for all stories
]


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Calculate vocabulary size (you'll need to adapt this based on your data)
# Assuming 'cleaned_stories' is a list of preprocessed stories (strings)
all_words = " ".join(cleaned_stories).split()
vocab_size = len(set(all_words)) + 1  # +1 for unknown words (usually denoted by 0)

# Encoder
encoder_inputs = Input(shape=(None, vocab_size))
encoder_lstm = LSTM(256, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, vocab_size))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [11]:
from sentence_transformers import SentenceTransformer

# Assuming 'cleaned_stories' contains the preprocessed stories:
combined_text = "\n\n".join(cleaned_stories)  # Join the stories with double newlines

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = [model.encode(paragraph) for paragraph in combined_text.split("\n\n")]